In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url='http://www.fia.com/events/fia-formula-1-world-championship/season-2015/formula-one'
response = requests.get(url,params={})

In [3]:
soup=BeautifulSoup(response.content)
events=soup.findAll('div',{'class':'event-item past'})

In [4]:
events

[<div class="event-item past">
 <div class="inner">
 <div class="date cell"> <div class="date-inner">
 <div class="from-date cell">
 <div class="day">15</div>
 <div class="month">Mar</div>
 </div>
 </div> </div>
 <div class="event cell"> <a href="/championship/events/fia-formula-one-world-championship/season-2015/grand-prix-australia"><div class="event-name cell">Grand Prix of Australia</div>
 <div class="event-location">ALBERT PARK</div></a> </div>
 <div class="classification cell"> <a class="link-label" href="/events/fia-formula-one-world-championship/season-2015/race-classification">Classification</a> </div>
 <div class="country cell"> <div><img alt="AUS" src="http://www.fia.com/sites/all/modules/contrib/countryicons_gosquared/flat-large/au.png" title="AUS"/> <div class="country-name">AUS</div>
 </div> </div> </div>
 </div>, <div class="event-item past">
 <div class="inner">
 <div class="date cell"> <div class="date-inner">
 <div class="from-date cell">
 <div class="day">29</div>
 <

In [5]:
!ls eventTiming/

AUS AUT BEL BHR CAN CHN ESP GBR HUN ITA JPN MCO MYS SGP


In [6]:
timing=[x.lower() for x in ['Official Classification','Provisional Classification','Preliminary Classification',
        'Fastest Laps','History Chart','Lap Chart','Lap Analysis','Pit Stop Summary','Best Sector Times'
        ,'Maximum Speeds','Official Starting Grid','Provisional Starting Grid','Qualifying Session',
        'Official Classification','Provisional Classification','Speed Trap',
        'Preliminary Classification','Lap Times','Provisionnal Classification','Provisionnal Starting Grid',
        'Speed Trap','Classification']]


In [8]:
import os
domain="http://www.fia.com"

import re

def download_file(url,f):
    response = re.urlopen(url)
    file = open("document.pdf", 'w')
    file.write(response.read())
    file.close()
    print("Completed")

for event in events[7:]:
    prev=''
    phase='R'
    gp=event.find('div',{'class':'event-name'}).text.replace('Grand Prix of ','').strip()
    c=event.find('div',{'class':'country-name'}).text.strip()
    l=event.find('div',{'class':'event-location'}).text.strip()
    u=event.find('div',{'class':'event cell'}).find('a')['href']
    
    print(gp,c,l)
    d='eventTiming/{}'.format(c)
    if not os.path.exists(d):
        os.makedirs(d) 
    url='{}{}'.format(domain,u)
    print(url)
    eventpage=requests.get(url)
    eventsoup=BeautifulSoup(eventpage.content)
    print(eventsoup.find('a',text='Event & Timing information')['href'])
    pdfsoup=BeautifulSoup(requests.get('{}{}'.format(domain,eventsoup.find('a',text='Event & Timing information')['href'])).content)
    middle=pdfsoup.find('div',{'class':'content'}).find('div',{'class':'middle'})

    for maybe in middle.findAll("a"):
        #print(maybe['href'])
        prepath=maybe.text.encode('ascii', 'ignore').strip()
        #print('fing path*{}*'.format(prepath))
        if prepath.lower() in timing:
            if (prev=='Official Starting Grid' and prepath=='Lap Times'): phase='P3'
            path='{} - {}'.format(phase,prepath)
            if prepath=='Provisional Starting Grid' or prepath=='Provisionnal Starting Grid':
                phase='Q'
            elif (phase=='Q' and prepath=='Maximum Speeds') or (phase=='Q' and (c=='MYS' or c=='CHN') and prepath=='Provisionnal Classification'):
                phase='P3'
            elif phase=='P3' and prepath=='Classification' :
                phase='P2'
            elif phase=='P2' and prepath=='Classification':
                phase='P1'
        else:
            path=prepath
        prev=prepath
        p='{d}/{path}.pdf'.format(d=d,path=path.replace('/','_'))

        #print(p)
        if not os.path.isfile(p):
            print("downloading",d,maybe.text.encode('ascii', 'ignore'))
            os.system('wget {url} -O "{p}"'.format(url='{}{}'.format(domain,maybe['href']),p=p))


(u'Austria', u'AUT', u'RED BULL RING - SPIELBERG')
http://www.fia.com/championship/events/fia-formula-one-world-championship/season-2015/grand-prix-austria
/events/fia-formula-one-world-championship/season-2015/event-timing-information-8
(u'Great Britain', u'GBR', u'SILVERSTONE GRAND PRIX')
http://www.fia.com/championship/events/fia-formula-one-world-championship/season-2015/grand-prix-great-britain
/events/fia-formula-one-world-championship/season-2015/event-timing-information-9
(u'Hungary', u'HUN', u'HUNGARORING')
http://www.fia.com/championship/events/fia-formula-one-world-championship/season-2015/grand-prix-hungary
/events/fia-formula-one-world-championship/season-2015/event-timing-information-11
(u'Belgium', u'BEL', u'SPA-FRANCORCHAMPS')
http://www.fia.com/championship/events/fia-formula-one-world-championship/season-2015/grand-prix-belgium
/events/fia-formula-one-world-championship/season-2015/event-timing-information-12
(u'Italy', u'ITA', u'MONZA')
http://www.fia.com/championshi

In [44]:
ls

01_australia_eng.pdf
02_malesia_eng.pdf
PDF scraper.ipynb
Pirelli2015MediaGuide.pdf
eventTiming/
pirelli_preview_australia_25311.pdf
pirelli_preview_china_25671.pdf
pirelli_preview_malaysia_25495.pdf
pirelli_review_australia_25422.pdf
pirelli_review_malaysia_25605.pdf
rsf1-presskit-2015-en_v6.pdf
ubs-formula-1-f1-race-strategy-briefing-infographic-kuala lumpur-malaysia.pdf
